<a href="https://colab.research.google.com/github/vngeno/E-COMMERCE-RECOMMENDATION-SYSTEM-/blob/main/Content_based_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [ ]:
!pip install wandb

In [ ]:
txt = pd.read_csv("/content/drive/Shareddrives/Team Stars/Data/df_clean.csv",index_col=0)
txt.head(2)

,id,name,brand,primarycategories,manufacturernumber,reviews_dorecommend,reviews_numhelpful,reviews_rating,reviews_text,reviews_title,reviews_username
0,AVpgNzjwLJeJML43Kpxn,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,HL-002619,True,1.0,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang
1,AVpgNzjwLJeJML43Kpxn,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,HL-002619,True,1.0,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG


In [ ]:
reviews = txt[['reviews_text', 'reviews_rating']]
reviews.head(2)

,reviews_text,reviews_rating
0,I order 3 of them and one of the item is bad q...,3
1,Bulk is always the less expensive way to go fo...,4


In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28332 entries, 0 to 28331
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviews_text    28332 non-null  object
 1   reviews_rating  28332 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 664.0+ KB


In [ ]:
rev = txt['reviews_text']

In [ ]:
rev.head(1)

0    I order 3 of them and one of the item is bad q...
Name: reviews_text, dtype: object

It is not possible to compute the similarity between any two reviews in their raw forms. To do this, we'll need to compute the word vectors of each review whereby word vectors are vectorized representation of words in a the reviews.

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
rev = rev.fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(rev)

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(28332, 10074)

In [ ]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[2000:2010]

['complaint',
 'complaints',
 'complants',
 'complement',
 'complete',
 'completed',
 'completely',
 'completion',
 'completly',
 'complex']

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosinesim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosinesim.shape

In [ ]:
cosinesim[1]

In [ ]:
#Construct a reverse map of indices and review titles
indices = pd.Series(txt.index, index=txt['name']).drop_duplicates()

In [ ]:
indices[:10]

In [ ]:
# Function that takes in product as input and outputs most similar products
def get_recommendations(name, cosine_sim=cosinesim):
    # Get the index of the product that matches the title
    idx = indices

    # Get the pairwsie similarity scores of all items with that product
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the items based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar products
    sim_scores = sim_scores[1:11]

    # Get the product indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar products
    return txt['name'].iloc[product_indices]


In [ ]:
txt.head(1)

In [ ]:
txt.name[1]